# ToS Mixedbread Embedding Creation

State-of-the-art sentence embeddings from mixedbread.ai. 

https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1.

In [1]:
# Packages
from sentence_transformers import SentenceTransformer
# NLTK for sentence tokenization
import nltk
nltk.download('punkt')

c:\Users\ijyli\anaconda3\envs\data_engineering_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ijyli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Load Model

In [2]:
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

## Function to Get Embeddings from Doc

In [ ]:
def get_doc_embeddings(filename):
    '''
    Get sentence embeddings from a document.
    '''
    # Read file
    with open('../Text Data/Terms of Service Corpus/text/' + filename, 'r') as f:
        doc = f.read()
    # Convert unicode to ascii
    doc = doc.encode('ascii', 'ignore').decode('ascii')
    # Parse company name by taking the first part of filename before underscore
    company_name = filename.split('_')[0]
    # Sentence tokenize the document
    sentences = nltk.sent_tokenize(doc)
    # Append company name to the beginning of each sentence
    sentences = [company_name + ' ' + sentence for sentence in sentences]
    # Encode sentences and return
    embeddings = model.encode(sentences)
    return embeddings
    

## Load files